<a href="https://colab.research.google.com/github/Ellena-F/Expense-tracker-and-bank-statement-reader/blob/master/read_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# install package
!pip install pdfminer
!pip install PyPDF2
!pip install tabula-py

     |████████████████████████████████| 4.2MB 2.8MB/s 
     |████████████████████████████████| 13.7MB 293kB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-cp36-none-any.whl size=6140069 sha256=adc934369b589346bf689f54a44abd452701f35160e2b5b2d05467d62be59534
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built pdfminer
     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=cd1c1ba12ab7fc4f94ef3ae225e49c6bb82579ed03bfa4b97a7d4e3f0554e35c
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
     |████████████████████████████████| 10.4MB 2.7MB/s 


In [4]:
# import package
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.collocations import *
from nltk.stem import *
from nltk.stem.porter import *
import seaborn as sns #basic charts
import matplotlib.pyplot as plt #plot setup
from wordcloud import WordCloud #word cloud
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import io
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
import tabula
import math

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
 April_Statement.csv	 'Getting started.pdf'	   reallynewApril_Statement.csv
 April_Statementnew.csv   newApril_Statement.csv  'SQL cheat sheet.docx'
 BankStatement		  read_statement.ipynb	   zt_sql_cheat_sheet.pdf


In [0]:
# import os # List all files in a directory using os.listdir

# basepath = '/content/drive/My Drive/BankStatement'
# name_list = []
# for entry in os.listdir(basepath):
#     if os.path.isfile(os.path.join(basepath, entry)):
#         name_list.append(entry)
# name_list.sort() # sort the file names

Tokenization


In [0]:
#set filepath
filepath='/content/drive/My Drive/BankStatement/Jan.pdf'
filepath4='/content/drive/My Drive/BankStatement/April.pdf'

FOR APRIL

In [7]:
#calculate page number for pdf file 
reader = PdfFileReader(open(filepath4, mode='rb' ))
n = reader.getNumPages() 
n

10

In [8]:
#read pdf file into df
rows4 = tabula.read_pdf(filepath4,
                       pages=("2-"+str(n)),
                       stream=True
                      #  ,encoding = 'ISO-8859-1'
                      ,columns=[36.95,92.72,303.9,408.44,515.88]
                       ,area=(180.61,36.95,749.29,574.99)
                       ,multiple_tables=True
                      #  silent=True
                      # #  pandas_options={
                      # #      'header': None}
                       )


Got stderr: May 03, 2020 1:12:37 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 03, 2020 1:12:37 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 03, 2020 1:12:37 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 03, 2020 1:12:37 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
May 03, 2020 1:12:37 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [0]:
# rows4[0]

In [0]:
# rows4[i]=rows4[2].dropna(axis='columns',thresh=rows4[2].shape[0]*0.1,how='all')
# rows4[0].columns=['Data',	'Transaction Details',	'Debits/Withdrawals',	'Credits/Deposits',	'Balance']
# rows4[5].dropna(axis='columns',thresh=rows4[5].shape[0]*0.05,how='all')

In [0]:
#drop column if na>0.05 of the number of rows
#reassign column name
for i in range(len(rows4)-1):
  rows4[i]=rows4[i].dropna(axis='columns',thresh=rows4[i].shape[0]*0.05,how='all')
  rows4[i].columns=['Date',	'Transaction Details',	'Debits/Withdrawals',	'Credits/Deposits',	'Balance']

In [9]:
rows4[0].head()

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
0,02 Jan,BALANCE BROUGHT FORWARD,NaN,NaN,107.16
1,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,NaN,100.00,NaN
2,NaN,TRANSFER FROM 002-331031-118,NaN,"1,000.00",NaN
3,NaN,my love WAI CHUN LAM 0573075,NaN,"1,000.00","2,207.16"
4,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465,296.89,NaN,NaN


In [9]:
#concatenate different page of pdf into 1 df
tryframe22=[]
for i in range(len(rows4)-1):
  tryframe22+=[rows4[i]]

trydf22=pd.concat(tryframe22)
trydf22
# tryframe=[rows4[0],rows4[1]]
# trydf2=pd.concat(tryframe)
# trydf2


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
0,NaN,02 Jan,BALANCE BROUGHT FORWARD,NaN,NaN,107.16
1,NaN,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,NaN,100.00,NaN
2,NaN,NaN,TRANSFER FROM 002-331031-118,NaN,"1,000.00",NaN
3,NaN,NaN,my love WAI CHUN LAM 0573075,NaN,"1,000.00","2,207.16"
4,NaN,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465,296.89,NaN,NaN
...,...,...,...,...,...,...
7,NaN,NaN,Transaction Total,"5,713.22","7,650.10",NaN
8,NaN,NaN,Transaction Number,98,68,NaN
9,NaN,NaN,END OF STA,TEMENT,NaN,NaN
10,NaN,NaN,237722_022TOA,NaN,NaN,NaN


In [0]:
trydf22=trydf22.reset_index(drop=True)

for i in range(1,len(trydf22)):

  if pd.isnull(trydf22['Debits/Withdrawals'][i])==True and pd.isnull(trydf22['Credits/Deposits'][i])==True:
    m=i-1
    trydf22['Transaction Details'][m]=trydf22['Transaction Details'][m]+' '+trydf22['Transaction Details'][i]


In [65]:
trydf22[:10]

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
1,NaN,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,NaN,100.00,NaN
2,NaN,NaN,TRANSFER FROM 002-331031-118,NaN,"1,000.00",NaN
3,NaN,NaN,my love WAI CHUN LAM 0573075,NaN,"1,000.00","2,207.16"
4,NaN,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,NaN,NaN
6,NaN,NaN,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,NaN,NaN
8,NaN,NaN,EFTPOS VISA AUD PAYPAL *MNGBAKERYPT 797577 402...,37,NaN,NaN
10,NaN,NaN,EFTPOS VISA AUD COLES 7842 471277 COLLINGWOOD AU,18.94,NaN,NaN
12,NaN,NaN,13JAN20 471277 13:40:13 COLES 7842 2% Cashbac...,NaN,0.38,"1,810.81"
14,NaN,15 Jan,EFTPOS VISA AUD WOOLWORTHS 3332 104312 FITZROYAU,33.95,NaN,NaN
16,NaN,NaN,EFTPOS VISA AUD WOOLWORTHS 3332 304798 FITZROYAU,1.63,NaN,NaN


In [0]:
#drop rows if both debit and credit amount are null
trydf22=trydf22.dropna(subset=['Debits/Withdrawals','Credits/Deposits'], how='all')


In [66]:
#fill null value to 0 
trydf22=trydf22.fillna(0)
trydf22

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
1,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100.00,0
2,0.0,0,TRANSFER FROM 002-331031-118,0,"1,000.00",0
3,0.0,0,my love WAI CHUN LAM 0573075,0,"1,000.00","2,207.16"
4,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0
6,0.0,0,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0
...,...,...,...,...,...,...
318,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,"1,043.04"
320,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,"1,001.00","2,044.04"
322,0.0,0,Transaction Total,"5,713.22","7,650.10",0
323,0.0,0,Transaction Number,98,68,0


In [67]:
# df111=trydf22
trydf22.tail()

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
318,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,"1,043.04"
320,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,"1,001.00","2,044.04"
322,0.0,0,Transaction Total,"5,713.22","7,650.10",0
323,0.0,0,Transaction Number,98,68,0
324,0.0,0,END OF STA 237722_022TOA,TEMENT,0,0


In [0]:
#define function to check unnecessary information
def check_tran (text):
  if text in ['Transaction Total','Transaction Number']:
    return 1
  else:
    return 0 

In [69]:
check_tran('Transaction')

0

In [70]:
trydf22['filter'] = trydf22['Transaction Details'].apply(check_tran)
trydf22

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter
1,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100.00,0,0
2,0.0,0,TRANSFER FROM 002-331031-118,0,"1,000.00",0,0
3,0.0,0,my love WAI CHUN LAM 0573075,0,"1,000.00","2,207.16",0
4,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0
6,0.0,0,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0
...,...,...,...,...,...,...,...
318,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,"1,043.04",0
320,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,"1,001.00","2,044.04",0
322,0.0,0,Transaction Total,"5,713.22","7,650.10",0,1
323,0.0,0,Transaction Number,98,68,0,1


In [71]:
trydf22=trydf22[trydf22['filter'] == 0]
trydf22=trydf22.reset_index(drop=True)
trydf22.tail()

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter
162,0.0,0,my love WAI CHUN LAM 0380243,0,"1,000.00","1,128.40",0
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0
164,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,"1,043.04",0
165,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,"1,001.00","2,044.04",0
166,0.0,0,END OF STA 237722_022TOA,TEMENT,0,0,0


In [0]:

# #remove rows with transaction total and number 
# remove_row=[]
# for i in range(len(df111)):
#   if 'Transaction Total' in df111.loc[i,'Transaction Details'] or'Transaction Number' in df111.loc[i,'Transaction Details'] :
#     remove_row.append(i)
# # remove_row    
# for each in remove_row:
#     df111=df111.drop(df111.index[each])
    
# df111=df111.reset_index(drop=True)


# # for i in range(len(df111)):
# #   if df111.loc[i,'Transaction Details']=='Transaction TotaNumber':
# #     df111=df111.drop(df111.index[i])

# #   if df111.loc[i,'Transaction Detailsdf111.loc[i,'Transaction Details']=='Transaction Total'']=='Transaction Number':
# #     df111=df111.drop(df111.index[i])
# df111.tail()

In [0]:
# for i in range(len(df111)):
#   if 'Transaction Number' in df111.loc[i,'Transaction Details']:
#     df111=df111.drop(df111.index[i])

In [0]:
#remove , from numeric value on debit column
#convert debit column value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Debits/Withdrawals'])==str:
    trydf22.loc[i,'Debits/Withdrawals']=float(trydf22.loc[i,'Debits/Withdrawals'].replace(',', ''))
  else:
    pass

In [0]:
#remove , from numeric value on credit column
#convert credit column value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Credits/Deposits'])==str:
   trydf22.loc[i,'Credits/Deposits']=float(trydf22.loc[i,'Credits/Deposits'].replace(',', ''))
  else:
    pass

In [0]:
#remove , from numeric value on balance column
#convert credit balance value to float
for i in range(len(trydf22)-1):
  if type(trydf22.loc[i,'Balance'])==str:
    ba=float(trydf22.loc[i,'Balance'].replace(',', ''))
    trydf22.loc[i,'Balance']=ba

  else:
    pass

In [75]:
#remove row if debits value is not numeric

trydf22=trydf22[trydf22['Debits/Withdrawals'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
# trytrydf=trytrydf[trytrydf['Credits/Deposits'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
# trytrydf=trytrydf[trytrydf['Balance'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
trydf22.tail()
# trytrydf.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter
161,0.0,0,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL...,0,1.5,0,0
162,0.0,0,my love WAI CHUN LAM 0380243,0,1000,1128.4,0
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0
164,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,1043.04,0
165,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,1001,2044.04,0


In [76]:
#reset index
trydf22=trydf22.reset_index(drop=True)
trydf22

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter
0,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0
1,0.0,0,TRANSFER FROM 002-331031-118,0,1000,0,0
2,0.0,0,my love WAI CHUN LAM 0573075,0,1000,2207.16,0
3,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0
4,0.0,0,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0
...,...,...,...,...,...,...,...
161,0.0,0,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL...,0,1.5,0,0
162,0.0,0,my love WAI CHUN LAM 0380243,0,1000,1128.4,0
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0
164,0.0,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,1043.04,0


#Categorise transactions

In [0]:
#Main categories and keyword for transaction
deb_keyword=['TRANSFER', 'WAI', 'ELLENA','IDR', 'AIRBNB','COLES','WOOLWORTHS','CHEMIST WAREHOUSE','ASIAN GROCER','CRUELTY FREE SHOP','SOURCE BULK FOOD',
         'PETSTOCK','PETBARN','VEGIE BAR','SUSHI','FINAS','MIN LOKAL CAFE','LOVING HUT','DABUEKS DONUTS','NEKO NEKO', 'FITZ CAFE','TERROR TWILIGHT',
         'GLASS DEN','MCDONALDS','JAPANESE','EASI','UBER EATS','UBER*EATS','HUNGRYPANDA','LORE PERFUMERY','MIND GAMES','RIOTGAMLOL','PRICELINE',
         'OFFICEWORKS','KMART','INSURE','ORIGINENERGY','ORIGIN','PAYPAL','DUMPLINGS','DAISO','MEDIBANK','REJECT SHOP','RIOTGAM*LOL','MZ MARKT',
         'BLUE CHILLIES','DONUTS','HINOKI','ORGANIC WHOLEFOODS']


deb_cat={'Transfer': ['TRANSFER', 'WAI', 'ELLENA'], 'Travelling': ['IDR', 'AIRBNB'], 
         'Grocery':['COLES','WOOLWORTHS','CHEMIST WAREHOUSE','ASIAN GROCER','CRUELTY FREE SHOP','SOURCE BULK FOOD','HINOKI','ORGANIC WHOLEFOODS'], 
         'Pet':['PETSTOCK','PETBARN'], 'Eat out':['VEGIE BAR','DONUTS','BLUE CHILLIES','MZ MARKT','SUSHI','FINAS','MIN LOKAL CAFE','LOVING HUT','DABUEKS DONUTS',
                                                  'NEKO NEKO', 'FITZ CAFE','TERROR TWILIGHT','GLASS DEN','MCDONALDS','JAPANESE','DUMPLINGS'], 
         'Take away':['EASI','UBER EATS','UBER*EATS','HUNGRYPANDA'], 'Leisure':['LORE PERFUMERY','MIND GAMES','RIOTGAMLOL','PRICELINE','RIOTGAM*LOL',
                                                                    'OFFICEWORKS'], 'HomeGoods':['KMART','DAISO','REJECT SHOP'], 'Insurance':['INSURE','MEDIBANK'], 
         'Bills':['ORIGINENERGY','ORIGIN'],
         'Online':['PAYPAL']}
cre_keyword=['TRANSFER', 'WAI', 'ELLENA','Cashback','2%']
cre_cat={'Transfer': ['TRANSFER', 'WAI', 'ELLENA'],'Cashback':['Cashback','2%']}

In [97]:
#add column for category and keyword
trydf22['keyword']='unknown'
trydf22['Category']='unknown'
trydf22

,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
0,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0,unknown,unknown
1,0.0,13 Jan,TRANSFER FROM 002-331031-118,0,1000,0,0,unknown,unknown
2,0.0,13 Jan,my love WAI CHUN LAM 0573075,0,1000,2207.16,0,unknown,unknown
3,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0,unknown,unknown
4,0.0,14 Jan,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0,unknown,unknown
...,...,...,...,...,...,...,...,...,...
161,0.0,30 Mar,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL...,0,1.5,0,0,unknown,unknown
162,0.0,30 Mar,my love WAI CHUN LAM 0380243,0,1000,1128.4,0,unknown,unknown
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0,unknown,unknown
164,0.0,31 Mar,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,1043.04,0,unknown,unknown


In [0]:
# trydf22.loc[165]

Date                                         01 Apr
Transaction Details    TRANSFER FROM 002-331031-118
Debits/Withdrawals                                0
Credits/Deposits                               1001
Balance                                     2044.04
filter                                            0
keyword                                     unknown
Category                                    unknown
Name: 165, dtype: object

In [0]:
# for word in deb_keyword:  
#   if trytrydf.loc[0]['Debits/Withdrawals']>0:
#     if word in trytrydf['Transaction Details'][0]:
#       trytrydf['keyword'][0]='word'
# trytrydf.head()


In [98]:
#Assign key word for debit record
for word in deb_keyword:
  for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Debits/Withdrawals']>0:
      if word in trydf22['Transaction Details'][i].upper():
        trydf22['keyword'][i]=word
      else:
        pass

trydf22.head()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:5]


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
0,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0,unknown,unknown
1,0.0,13 Jan,TRANSFER FROM 002-331031-118,0,1000,0,0,unknown,unknown
2,0.0,13 Jan,my love WAI CHUN LAM 0573075,0,1000,2207.16,0,unknown,unknown
3,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0,AIRBNB,unknown
4,0.0,14 Jan,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0,MZ MARKT,unknown


In [0]:
# trydf22.tail()

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
161,0,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL,0,1.5,0,0,unknown,unknown
162,0,my love WAI CHUN LAM 0380243,0,1000,1128.4,0,unknown,unknown
163,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930,46.08,0,0,0,HungryPanda,unknown
164,0,EFTPOS VISA AUD PIN*189 HungryPanda 381998,39.28,0,1043.04,0,HungryPanda,unknown
165,01 Apr,TRANSFER FROM 002-331031-118,0,1001,2044.04,0,unknown,unknown


In [0]:
# cre_keyword

['TRANSFER', 'WAI', 'ELLENA', 'Cashback', '2%']

In [80]:
# for word in cre_keyword:

#   if trydf22.loc[165]['Credits/Deposits']>0:
#     if word in trydf22['Transaction Details'][165]:
#       print(word)
#       # trydf22['keyword'][i]=word


TRANSFER


In [99]:
#Assign key work for credit record
for word in cre_keyword:
  for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Credits/Deposits']>0:
      if word in trydf22['Transaction Details'][i].upper():
        trydf22['keyword'][i]=word
      else:
        pass

trydf22.head()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:5]


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
0,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0,unknown,unknown
1,0.0,13 Jan,TRANSFER FROM 002-331031-118,0,1000,0,0,TRANSFER,unknown
2,0.0,13 Jan,my love WAI CHUN LAM 0573075,0,1000,2207.16,0,WAI,unknown
3,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0,AIRBNB,unknown
4,0.0,14 Jan,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0,MZ MARKT,unknown


Assign category

In [100]:


# for cate in cre_cat:
#   for i in range(len(trytrydf)-1):
#     if trytrydf.loc[i]['Credits/Deposits']>0:
#       if cate in trytrydf['keyword'][i]:
#         trytrydf['Category'][i]=cate
#       else:
#         pass


# for cate in cre_cat:
#   for i in range(len(trytrydf)-1):
#     if trytrydf.loc[i]['Credits/Deposits']>0:
#       if cate in trytrydf['keyword'][i]:
#         trytrydf['Category'][i]=cate
#       else:
#         pass

for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Credits/Deposits']>0:

      for key in cre_cat.keys():
        for value in cre_cat[key]:
          if value in trydf22.loc[i]['keyword']:
            trydf22['Category'][i]=key
trydf22.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:8]


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
161,0.0,30 Mar,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL...,0,1.5,0,0,2%,Cashback
162,0.0,30 Mar,my love WAI CHUN LAM 0380243,0,1000,1128.4,0,WAI,Transfer
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0,HUNGRYPANDA,unknown
164,0.0,31 Mar,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,1043.04,0,HUNGRYPANDA,unknown
165,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,1001,2044.04,0,unknown,unknown


In [101]:
for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Debits/Withdrawals']>0:

      for key in deb_cat.keys():
        for value in deb_cat[key]:
          if value in trydf22.loc[i]['keyword']:
            trydf22['Category'][i]=key
trydf22.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:7]


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
161,0.0,30 Mar,26MAR20 661082 18:32:27 PETSTOCK CLIFTON HILL...,0,1.5,0,0,2%,Cashback
162,0.0,30 Mar,my love WAI CHUN LAM 0380243,0,1000,1128.4,0,WAI,Transfer
163,0.0,31 Mar,EFTPOS VISA AUD PIN*916 HungryPanda 582930 PRA...,46.08,0,0,0,HUNGRYPANDA,Take away
164,0.0,31 Mar,EFTPOS VISA AUD PIN*189 HungryPanda 381998 PRA...,39.28,0,1043.04,0,HUNGRYPANDA,Take away
165,0.0,01 Apr,TRANSFER FROM 002-331031-118 CLOSING BALANCE,0,1001,2044.04,0,unknown,unknown


In [102]:
#assign empty date value to last record date
trydf22['Date']=trydf22['Date'].replace(0, np.nan).ffill()
trydf22.head()


,Unnamed: 0,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
0,0.0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0,unknown,unknown
1,0.0,13 Jan,TRANSFER FROM 002-331031-118,0,1000,0,0,TRANSFER,Transfer
2,0.0,13 Jan,my love WAI CHUN LAM 0573075,0,1000,2207.16,0,WAI,Transfer
3,0.0,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465 SUR...,296.89,0,0,0,AIRBNB,Travelling
4,0.0,14 Jan,EFTPOS VISA AUD MZ MARKT P/L 188816 COLLINGWOO...,43.9,0,0,0,MZ MARKT,Eat out


In [0]:
#output file
# trytrydf.tail()
trydf22.to_csv('finalApril_Statement.csv')
# !cp data.csv "drive/My Drive/"
!cp finalApril_Statement.csv "/content/drive/My Drive"

In [0]:
trydf22.loc[73]

Date                                                       25 Feb
Transaction Details    EFTPOS VISA AUD PIN*817 HungryPanda 441270
Debits/Withdrawals                                          36.95
Credits/Deposits                                                0
Balance                                                     82.94
Category                                                   unknow
keyword                                                    unknow
Name: 73, dtype: object

In [0]:
#Assign key work for debit record
for word in deb_keyword:
  for i in range(len(trydf22)-1):
    if trydf22.loc[i]['Debits/Withdrawals']>0:
      if word in trydf22['Transaction Details'][i]:
        trydf22['keyword'][i]=word
      else:
        pass

trydf22.head()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:5]


,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,filter,keyword,Category
0,13 Jan,$100 BONUS - Enjoy MFI Bonus 002376515090,0,100,0,0,unknown,unknown
1,13 Jan,TRANSFER FROM 002-331031-118,0,1000,0,0,TRANSFER,Transfer
2,13 Jan,my love WAI CHUN LAM 0573075,0,1000,2207.16,0,WAI,Transfer
3,14 Jan,EFTPOS VISA AUD AIRBNB * HMTMFA9F5H 462465,296.89,0,0,0,AIRBNB,Travelling
4,14 Jan,EFTPOS VISA AUD MZ MARKT P/L 188816,43.9,0,0,0,unknown,unknown


In [0]:
# trydf22.loc[73]['Debits/Withdrawals']>0

True

In [0]:
# deb_keyword

In [0]:
# for word in deb_keyword:
#   if word in trydf22['Transaction Details'][73]:
#     print ('True')
#   else: 
#     print ('f')

# **FOR JAN**





In [0]:
import tabula
# reading table using tabula
rows = tabula.read_pdf(filepath,
                       pages=("2-3"),
                       stream=True
                      #  ,encoding = 'ISO-8859-1'
                       ,area=(180.61,30.64,749.29,574.99),
                       multiple_tables=True
                      #  silent=True
                      # #  pandas_options={
                      # #      'header': None}
                       )
rows
# converting to list
# rows = rows.values.tolist()

Got stderr: Apr 14, 2020 1:31:08 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Apr 14, 2020 1:31:08 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 14, 2020 1:31:08 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 14, 2020 1:31:08 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Apr 14, 2020 1:31:08 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



[      Date  ...   Balance
 0      NaN  ...      0.00
 1   18 Oct  ...  2,000.00
 2   21 Oct  ...       NaN
 3      NaN  ...       NaN
 4      NaN  ...       NaN
 5      NaN  ...       NaN
 6      NaN  ...       NaN
 7      NaN  ...       NaN
 8      NaN  ...       NaN
 9      NaN  ...       NaN
 10     NaN  ...       NaN
 11     NaN  ...  3,626.48
 12  22 Oct  ...       NaN
 13     NaN  ...       NaN
 14     NaN  ...       NaN
 15     NaN  ...       NaN
 16     NaN  ...  3,091.19
 17     NaN  ...       NaN
 18  24 Oct  ...  2,884.71
 19     NaN  ...       NaN
 20  05 Nov  ...  5,997.71
 21  08 Nov  ...  2,884.71
 22  21 Nov  ...       NaN
 23     NaN  ...       NaN
 24     NaN  ...  2,875.08
 25     NaN  ...       NaN
 26  22 Nov  ...  2,833.66
 27     NaN  ...       NaN
 28  26 Nov  ...       NaN
 29     NaN  ...       NaN
 30     NaN  ...       NaN
 31     NaN  ...       NaN
 32     NaN  ...       NaN
 33     NaN  ...       NaN
 34     NaN  ...       NaN
 35     NaN  ...       NaN
 

In [0]:
rows[1]

In [0]:
df1=rows[0]
df2=rows[1]
df2

,Date,Transaction Details Debits/Withdrawals,Unnamed: 0,Credits/Deposits,Balance
0,NaN,EFTPOS VISA IDR 236000.00 CANGGU DELI PT HO,24.75,NaN,488.72
1,NaN,2847511847,NaN,NaN,NaN
2,02 Dec,TRANSFER TO 002-331031-118 Lost card 178.00,NaN,NaN,NaN
3,NaN,01DEC19 ATMA896 08:46:46 VISA IDR 1250000.00 1...,NaN,NaN,NaN
4,NaN,WANAMART 01 PTUGDPS 5246571847,NaN,NaN,NaN
5,NaN,EFTPOS VISA IDR 310000.00 COFFEE CARTEL 502953,32.56,NaN,NaN
6,NaN,1847,NaN,NaN,NaN
7,NaN,EFTPOS VISA IDR 282975.00 KEMBALI 470942,29.73,NaN,NaN
8,NaN,1847,NaN,NaN,NaN
9,NaN,"EFTPOS VISA IDR94500.00 GUARDIAN,PETITENGET",9.93,NaN,NaN


In [0]:
df2.columns = ['Date',	'Transaction Details',	'Debits/Withdrawals',	'Credits/Deposits',	'Balance']
df2

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
0,NaN,EFTPOS VISA IDR 236000.00 CANGGU DELI PT HO,24.75,NaN,488.72
1,NaN,2847511847,NaN,NaN,NaN
2,02 Dec,TRANSFER TO 002-331031-118 Lost card 178.00,NaN,NaN,NaN
3,NaN,01DEC19 ATMA896 08:46:46 VISA IDR 1250000.00 1...,NaN,NaN,NaN
4,NaN,WANAMART 01 PTUGDPS 5246571847,NaN,NaN,NaN
5,NaN,EFTPOS VISA IDR 310000.00 COFFEE CARTEL 502953,32.56,NaN,NaN
6,NaN,1847,NaN,NaN,NaN
7,NaN,EFTPOS VISA IDR 282975.00 KEMBALI 470942,29.73,NaN,NaN
8,NaN,1847,NaN,NaN,NaN
9,NaN,"EFTPOS VISA IDR94500.00 GUARDIAN,PETITENGET",9.93,NaN,NaN


In [0]:
frames=[df1,df2]
df = pd.concat(frames)
df

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
0,NaN,OPENING BALANCE,NaN,NaN,0.00
1,18 Oct,TRANSFER FROM 002-331031-118,NaN,"2,000.00","2,000.00"
2,21 Oct,EFTPOS VISA AUD AUTHORIZATION 552628 SYDNEY,301.72,NaN,NaN
3,NaN,AU,NaN,NaN,NaN
4,NaN,EFTPOS VISA AUD AUTHORIZATION 649508 SYDNEY,55.04,NaN,NaN
...,...,...,...,...,...
15,NaN,"Transaction Total 7,082.60",NaN,"7,189.76",NaN
16,NaN,Transaction Number,27,6,NaN
17,NaN,END OF STATEMENT,NaN,NaN,NaN
18,NaN,237722_022TOA,NaN,NaN,NaN


In [0]:
df=df.dropna(subset=['Debits/Withdrawals','Credits/Deposits'], how='all')
df

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
1,18 Oct,TRANSFER FROM 002-331031-118,NaN,"2,000.00","2,000.00"
2,21 Oct,EFTPOS VISA AUD AUTHORIZATION 552628 SYDNEY,301.72,NaN,NaN
4,NaN,EFTPOS VISA AUD AUTHORIZATION 649508 SYDNEY,55.04,NaN,NaN
6,NaN,EFTPOS VISA AUD AUTHORIZATION 0459SYDNEY,16.76,NaN,NaN
8,NaN,EFTPOS VISA AUD AUTHORIZATION 649492 SYDNEY,16.76,NaN,NaN
10,NaN,CREDIT VOUCHER 18/10/19 AUTHORIZATION,NaN,16.76,NaN
11,NaN,Tommy WAI CHUN LAM 0621821,NaN,"2,000.00","3,626.48"
12,22 Oct,EFTPOS VISA AUD GCS AGODA INTERNATIONA 153994,384.92,NaN,NaN
14,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 154659,92.32,NaN,NaN
16,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 415504,58.05,NaN,"3,091.19"


In [0]:
#  len(df.index)

#  df[:-3] #drop bottom 3 
 dftry= df[:-2]
 

In [0]:
dftry

In [0]:
len(dftry.index)

31

In [0]:
dftry=dftry.reset_index(drop=True)
dftry

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance
0,18 Oct,TRANSFER FROM 002-331031-118,NaN,"2,000.00","2,000.00"
1,21 Oct,EFTPOS VISA AUD AUTHORIZATION 552628 SYDNEY,301.72,NaN,NaN
2,NaN,EFTPOS VISA AUD AUTHORIZATION 649508 SYDNEY,55.04,NaN,NaN
3,NaN,EFTPOS VISA AUD AUTHORIZATION 0459SYDNEY,16.76,NaN,NaN
4,NaN,EFTPOS VISA AUD AUTHORIZATION 649492 SYDNEY,16.76,NaN,NaN
5,NaN,CREDIT VOUCHER 18/10/19 AUTHORIZATION,NaN,16.76,NaN
6,NaN,Tommy WAI CHUN LAM 0621821,NaN,"2,000.00","3,626.48"
7,22 Oct,EFTPOS VISA AUD GCS AGODA INTERNATIONA 153994,384.92,NaN,NaN
8,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 154659,92.32,NaN,NaN
9,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 415504,58.05,NaN,"3,091.19"


In [0]:
# range(len(dftry.index))

range(0, 31)

In [0]:
# for i in range(len(dftry.index)):
#   print (pd.isnull(dftry.iloc[i,2]))

False
False
False
False
False
True
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False


In [0]:
# print(dftry.iloc[5,2])
# pd.isnull(dftry.iloc[5,2])


nan


True

In [0]:
dftry['Type'] = 'Unknown'
#categories credit and debit
for i in range(len(dftry.index)):
  if pd.isnull(dftry.iloc[i,2])==False:
    dftry['Type'][i]='Debit'
  if pd.isnull(dftry.iloc[i,2])==True:
    dftry['Type'][i]='Credit'






In [0]:
dftry

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,Type
0,18 Oct,TRANSFER FROM 002-331031-118,NaN,"2,000.00","2,000.00",Credit
1,21 Oct,EFTPOS VISA AUD AUTHORIZATION 552628 SYDNEY,301.72,NaN,NaN,Debit
2,NaN,EFTPOS VISA AUD AUTHORIZATION 649508 SYDNEY,55.04,NaN,NaN,Debit
3,NaN,EFTPOS VISA AUD AUTHORIZATION 0459SYDNEY,16.76,NaN,NaN,Debit
4,NaN,EFTPOS VISA AUD AUTHORIZATION 649492 SYDNEY,16.76,NaN,NaN,Debit
5,NaN,CREDIT VOUCHER 18/10/19 AUTHORIZATION,NaN,16.76,NaN,Credit
6,NaN,Tommy WAI CHUN LAM 0621821,NaN,"2,000.00","3,626.48",Credit
7,22 Oct,EFTPOS VISA AUD GCS AGODA INTERNATIONA 153994,384.92,NaN,NaN,Debit
8,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 154659,92.32,NaN,NaN,Debit
9,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 415504,58.05,NaN,"3,091.19",Debit


In [0]:
import tokenize

In [0]:
# def categorize(particular):
#     # tokenizing
#     word_list = nltk.tokenize.word_tokenize(particular)
    
#     # multiple similar statements
#     if keyword in word_list:
#         return category

# # # iterate over all transactions
# # for i in range(len(dftry.index)):
# #     # assign category

# #     dftry['category'][i] = categorize(dftry['particular'])

# for i in range(5):    
#     a.append(i)

In [0]:
# words1.append(nltk.tokenize.word_tokenize(dftry['Transaction Details'][0]))
# words1

['TRANSFER',
 'FROM',
 '002-331031-118',
 'Try',
 'EFTPOS',
 'VISA',
 'AUD',
 'AUTHORIZATION',
 '649508',
 'SYDNEY',
 ['TRANSFER', 'FROM', '002-331031-118']]

In [0]:
cat_key=['TRANSFER','WAI','ELLENA','IDR']

In [0]:
# dftry['Transaction Details'][10]
dftry['Category']='unknow'


In [0]:
for word in cat_key:
  for i in range(len(dftry.index)):
    if word in dftry['Transaction Details'][i]:
      dftry['Category'][i]=word

dftry


In [0]:
# for i in range(len(dftry.index)):
#     # assign category
#   words=words+nltk.tokenize.word_tokenize(dftry['Transaction Details'][i])
# words

In [0]:
# import collections
# from collections import Counter
# Counter(words)

In [0]:
nltk.tokenize.word_tokenize(dftry['Transaction Details'][2])+nltk.tokenize.word_tokenize(dftry['Transaction Details'][1])

['EFTPOS',
 'VISA',
 'AUD',
 'AUTHORIZATION',
 '649508',
 'SYDNEY',
 'EFTPOS',
 'VISA',
 'AUD',
 'AUTHORIZATION',
 '552628',
 'SYDNEY']

In [0]:
words=[]

In [0]:
type(dftry['Transaction Details'][0])

str

In [0]:
pd.isnull(df.iloc[5,2])== False

False

In [0]:
dftry

,Date,Transaction Details,Debits/Withdrawals,Credits/Deposits,Balance,Type,Category
0,18 Oct,TRANSFER FROM 002-331031-118,NaN,"2,000.00","2,000.00",Credit,TRANSFER
1,21 Oct,EFTPOS VISA AUD AUTHORIZATION 552628 SYDNEY,301.72,NaN,NaN,Debit,unknow
2,NaN,EFTPOS VISA AUD AUTHORIZATION 649508 SYDNEY,55.04,NaN,NaN,Debit,unknow
3,NaN,EFTPOS VISA AUD AUTHORIZATION 0459SYDNEY,16.76,NaN,NaN,Debit,unknow
4,NaN,EFTPOS VISA AUD AUTHORIZATION 649492 SYDNEY,16.76,NaN,NaN,Debit,unknow
5,NaN,CREDIT VOUCHER 18/10/19 AUTHORIZATION,NaN,16.76,NaN,Credit,unknow
6,NaN,Tommy WAI CHUN LAM 0621821,NaN,"2,000.00","3,626.48",Credit,WAI
7,22 Oct,EFTPOS VISA AUD GCS AGODA INTERNATIONA 153994,384.92,NaN,NaN,Debit,unknow
8,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 154659,92.32,NaN,NaN,Debit,unknow
9,NaN,EFTPOS VISA AUD GCS AGODA INTERNATIONA 415504,58.05,NaN,"3,091.19",Debit,unknow
